## Imports

In [65]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [137]:
!pip install --upgrade kaggle
!kaggle competitions download -c titanic

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [138]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.1.5
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [140]:
CLOUD = True

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "umutakta"
    os.environ['KAGGLE_KEY']      = "a471e5474add8d57cadac3cc5ff2b1d6"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../datasets/titanic/"

DATA_PATH

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 10.7MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 56.1MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.18MB/s]


'./'

## Load data

In [142]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [143]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [144]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [145]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda x: x.split(",")[1].split(".")[0].strip()

# YOUR CODE HERE


df['Title']      = df['Name'].map(get_Title_from_Name )
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [146]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [147]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [148]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df["Title"] = df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)

In [149]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [150]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [172]:
df_test['Cabin']

df.Cabin.fillna(0, inplace=True) #We named Null cells as 'Empty'
df_test.Cabin.fillna(0, inplace=True) #We named Null cells as 'Empty'

df.loc[df["Cabin"] != 0, "Cabin"] = 1
df_test.loc[df_test["Cabin"] != 0, "Cabin" ] =1




# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [173]:
x = df.drop(columns=["Survived", 'Name', 'Ticket']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket']) # # X_TEST DATA (NEW DATA)

In [174]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [175]:

num_4_treeModels = pipeline.Pipeline(steps=[('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)), 
                                            ])

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro



ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [176]:
len(num_4_treeModels)

1

In [177]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [178]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [179]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": GradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier() }


# YOUR CODE HERE


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [180]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [181]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

Learning rate set to 0.008911
0:	learn: 0.6874585	total: 1.33ms	remaining: 1.33s
1:	learn: 0.6817921	total: 2.68ms	remaining: 1.33s
2:	learn: 0.6768098	total: 4.02ms	remaining: 1.33s
3:	learn: 0.6720448	total: 5.31ms	remaining: 1.32s
4:	learn: 0.6669187	total: 6.63ms	remaining: 1.32s
5:	learn: 0.6622295	total: 7.93ms	remaining: 1.31s
6:	learn: 0.6573380	total: 9.26ms	remaining: 1.31s
7:	learn: 0.6524357	total: 10.6ms	remaining: 1.31s
8:	learn: 0.6468926	total: 11.9ms	remaining: 1.31s
9:	learn: 0.6418077	total: 13.3ms	remaining: 1.31s
10:	learn: 0.6366277	total: 14.6ms	remaining: 1.31s
11:	learn: 0.6317214	total: 15.9ms	remaining: 1.31s
12:	learn: 0.6279207	total: 17.2ms	remaining: 1.3s
13:	learn: 0.6242446	total: 18ms	remaining: 1.27s
14:	learn: 0.6198698	total: 19.3ms	remaining: 1.27s
15:	learn: 0.6161807	total: 20.3ms	remaining: 1.25s
16:	learn: 0.6115137	total: 21.5ms	remaining: 1.24s
17:	learn: 0.6074706	total: 22.8ms	remaining: 1.24s
18:	learn: 0.6037416	total: 24.1ms	remaining: 1

,Model,Accuracy,Bal Acc.,Time
1,XGBoost,87.150838,85.494071,0.073597
2,CatBoost,86.592179,84.769433,1.628481
3,Skl GBM,86.033520,84.044796,0.135257
4,Skl HistGBM,86.033520,84.044796,0.139745
5,AdaBoost,83.798883,83.036891,0.121216
6,LightGBM,83.798883,82.766798,0.063219
7,Random Forest,82.681564,80.777339,0.239276
8,Extra Trees,80.446927,78.959157,0.187817
9,Decision Tree,77.094972,76.501976,0.023062


In [182]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [183]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in tree_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


Görüntülenen çıkış son 5000 satıra kısaltıldı.
4:	learn: 0.6664729	total: 6.96ms	remaining: 1.39s
5:	learn: 0.6604514	total: 8.39ms	remaining: 1.39s
6:	learn: 0.6548378	total: 9.84ms	remaining: 1.4s
7:	learn: 0.6495250	total: 11.2ms	remaining: 1.39s
8:	learn: 0.6440020	total: 12.6ms	remaining: 1.39s
9:	learn: 0.6386612	total: 14ms	remaining: 1.39s
10:	learn: 0.6342413	total: 15.4ms	remaining: 1.38s
11:	learn: 0.6300642	total: 17.1ms	remaining: 1.41s
12:	learn: 0.6246508	total: 18.7ms	remaining: 1.42s
13:	learn: 0.6197030	total: 20.5ms	remaining: 1.44s
14:	learn: 0.6144734	total: 22ms	remaining: 1.44s
15:	learn: 0.6095679	total: 23.4ms	remaining: 1.44s
16:	learn: 0.6047602	total: 25.4ms	remaining: 1.47s
17:	learn: 0.6005065	total: 26.9ms	remaining: 1.47s
18:	learn: 0.5960359	total: 28.3ms	remaining: 1.46s
19:	learn: 0.5915042	total: 30.4ms	remaining: 1.49s
20:	learn: 0.5872012	total: 31.8ms	remaining: 1.48s
21:	learn: 0.5828862	total: 33.3ms	remaining: 1.48s
22:	learn: 0.5792216	total: 

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.838384,81.703576,1.441082
2,Skl HistGBM,83.838384,81.703576,1.450693
3,CatBoost,83.389450,81.173905,17.224122
4,XGBoost,83.164983,80.716135,0.812417
5,LightGBM,82.154882,80.392580,0.702792
6,Random Forest,81.930415,80.430927,2.247535
7,AdaBoost,81.032548,79.922826,1.237712
8,Extra Trees,80.695847,79.318857,1.889848
9,Decision Tree,79.124579,78.154060,0.243834


In [186]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [187]:
best_model = tree_classifiers["XGBoost"]

# Fit best model with all data
best_model.fit(x,y)



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('xgbclassifier', XGBClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [188]:
test_pred = best_model.predict(x_test)


In [189]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [190]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [191]:
sub.to_csv("sub.csv")

In [169]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

100% 2.77k/2.77k [00:10<00:00, 265B/s]
Successfully submitted to Titanic - Machine Learning from Disaster